# 04 abr 2018

O objetivo de hoje é 

- Remontar o dataset de candidatos (2004 - 2016) com as informações do TSE
- Tratar os dados
- Popular as tabelas auxiliares com as informações do dataset
  - Estados
  - Cidades
  - Nacionalidades
  - Graus de Instrução
  - Ocupações
  - Eleições
  - Cargos
  - Partidos
  - Situações de uma Candidatura
  - Resultado de uma Candidatura

## Re-construção do dataset de Histórico de Campanhas

In [34]:
import pandas as pd
import numpy as np
import os
import urllib
import zipfile
import glob

from tempfile import mkdtemp
from utils.download import download

Definição de constantes a serem utilizadas ao longo do código

In [35]:
# Diretório temporário
# DIRETORIO_TEMPORARIO = mkdtemp()
DIRETORIO_TEMPORARIO = os.path.join('../temp')

# Nome e endereço dos arquivos do diretório do TSE
PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'
URL_CANDIDATURAS = 'http://agencia.tse.jus.br/estatistica/sead/odsele/consulta_cand/'

# Setup de configuração para o arquivo de saída (nome do dataset)
HOJE = pd.datetime.today().date()
ARQUIVO_SAIDA_CANDIDATURAS = HOJE.isoformat() + '-candidaturas-tse.xz'
DATASET_SAIDA_CANDIDATURAS = os.path.join('../data', ARQUIVO_SAIDA_CANDIDATURAS)

Como os arquivos das candidaturas anteriores a 2004 ainda apresentam incosistência, vamos realizar o download de todos os anos a partir de 2004 até a eleição anterior ao ano de processamento deste algoritmo

In [36]:
anos_eleitorais = [str(year) for year in (range(2004, HOJE.year - 1, 2))]

In [37]:
for ano in anos_eleitorais:
    nome_arquivo = '{}{}.zip'.format(PREFIXO_ARQUIVO_CANDIDATURAS, ano)
    url_arquivo = URL_CANDIDATURAS + nome_arquivo
    arquivo_saida = os.path.join(DIRETORIO_TEMPORARIO, nome_arquivo)
    
    # urllib.request.urlretrieve(url_arquivo, arquivo_saida)
    download(url_arquivo, arquivo_saida)

    print('Arquivos de ' + ano + ' baixados com sucesso!')

[██████████████████████████████████████████████████]
Arquivos de 2004 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2006 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2008 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2010 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2012 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2014 baixados com sucesso!
[██████████████████████████████████████████████████]
Arquivos de 2016 baixados com sucesso!


Como os arquivos baixados estão no formato .zip, precisamos extrair seu conteúdo para o diretório temporário

In [38]:
for ano in anos_eleitorais:
    arquivo = PREFIXO_ARQUIVO_CANDIDATURAS + ano + '.zip'
    path = os.path.join(DIRETORIO_TEMPORARIO, arquivo)
    
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(DIRETORIO_TEMPORARIO)
    zip_ref.close()

Agora temos arquivos `.txt` divididos por ano e por estado, para explorarmos.

Como os arquivos não contém _header_, descritos no arquivo LEIAME.pdf, precisaremos inserir manualmente.

In [39]:
header_consulta_cand_ate2010 = [
    "DATA_GERACAO",
    "HORA_GERACAO",
    "ANO_ELEICAO",
    "NUM_TURNO",  # (*)
    "DESCRICAO_ELEICAO",  # (*)
    "SIGLA_UF",
    "SIGLA_UE",  # (*)
    "DESCRICAO_UE",
    "CODIGO_CARGO",  # (*)
    "DESCRICAO_CARGO",
    "NOME_CANDIDATO",
    "SEQUENCIAL_CANDIDATO",  # (*)
    "NUMERO_CANDIDATO",
    "CPF_CANDIDATO",
    "NOME_URNA_CANDIDATO",
    "COD_SITUACAO_CANDIDATURA",
    "DES_SITUACAO_CANDIDATURA",
    "NUMERO_PARTIDO",
    "SIGLA_PARTIDO",
    "NOME_PARTIDO",
    "CODIGO_LEGENDA",
    "SIGLA_LEGENDA",
    "COMPOSICAO_LEGENDA",
    "NOME_LEGENDA",
    "CODIGO_OCUPACAO",
    "DESCRICAO_OCUPACAO",
    "DATA_NASCIMENTO",
    "NUM_TITULO_ELEITORAL_CANDIDATO",
    "IDADE_DATA_ELEICAO",
    "CODIGO_SEXO",
    "DESCRICAO_SEXO",
    "COD_GRAU_INSTRUCAO",
    "DESCRICAO_GRAU_INSTRUCAO",
    "CODIGO_ESTADO_CIVIL",
    "DESCRICAO_ESTADO_CIVIL",
    "CODIGO_NACIONALIDADE",
    "DESCRICAO_NACIONALIDADE",
    "SIGLA_UF_NASCIMENTO",
    "CODIGO_MUNICIPIO_NASCIMENTO",
    "NOME_MUNICIPIO_NASCIMENTO",
    "DESPESA_MAX_CAMPANHA",
    "COD_SIT_TOT_TURNO",
    "DESC_SIT_TOT_TURNO",
]

In [40]:
header_consulta_cand_em2012 = [
    "DATA_GERACAO",
    "HORA_GERACAO",
    "ANO_ELEICAO",
    "NUM_TURNO",  # (*)
    "DESCRICAO_ELEICAO",  # (*)
    "SIGLA_UF",
    "SIGLA_UE",  # (*)
    "DESCRICAO_UE",
    "CODIGO_CARGO",  # (*)
    "DESCRICAO_CARGO",
    "NOME_CANDIDATO",
    "SEQUENCIAL_CANDIDATO",  # (*)
    "NUMERO_CANDIDATO",
    "CPF_CANDIDATO",
    "NOME_URNA_CANDIDATO",
    "COD_SITUACAO_CANDIDATURA",
    "DES_SITUACAO_CANDIDATURA",
    "NUMERO_PARTIDO",
    "SIGLA_PARTIDO",
    "NOME_PARTIDO",
    "CODIGO_LEGENDA",
    "SIGLA_LEGENDA",
    "COMPOSICAO_LEGENDA",
    "NOME_LEGENDA",
    "CODIGO_OCUPACAO",
    "DESCRICAO_OCUPACAO",
    "DATA_NASCIMENTO",
    "NUM_TITULO_ELEITORAL_CANDIDATO",
    "IDADE_DATA_ELEICAO",
    "CODIGO_SEXO",
    "DESCRICAO_SEXO",
    "COD_GRAU_INSTRUCAO",
    "DESCRICAO_GRAU_INSTRUCAO",
    "CODIGO_ESTADO_CIVIL",
    "DESCRICAO_ESTADO_CIVIL",
    "CODIGO_NACIONALIDADE",
    "DESCRICAO_NACIONALIDADE",
    "SIGLA_UF_NASCIMENTO",
    "CODIGO_MUNICIPIO_NASCIMENTO",
    "NOME_MUNICIPIO_NASCIMENTO",
    "DESPESA_MAX_CAMPANHA",
    "COD_SIT_TOT_TURNO",
    "DESC_SIT_TOT_TURNO",
    "NM_EMAIL",
]

In [41]:
header_consulta_cand_pos2014 = [
    "DATA_GERACAO",
    "HORA_GERACAO",
    "ANO_ELEICAO",
    "NUM_TURNO",  # (*)
    "DESCRICAO_ELEICAO",  # (*)
    "SIGLA_UF",
    "SIGLA_UE",  # (*)
    "DESCRICAO_UE",
    "CODIGO_CARGO",  # (*)
    "DESCRICAO_CARGO",
    "NOME_CANDIDATO",
    "SEQUENCIAL_CANDIDATO",  # (*)
    "NUMERO_CANDIDATO",
    "CPF_CANDIDATO",
    "NOME_URNA_CANDIDATO",
    "COD_SITUACAO_CANDIDATURA",
    "DES_SITUACAO_CANDIDATURA",
    "NUMERO_PARTIDO",
    "SIGLA_PARTIDO",
    "NOME_PARTIDO",
    "CODIGO_LEGENDA",
    "SIGLA_LEGENDA",
    "COMPOSICAO_LEGENDA",
    "NOME_LEGENDA",
    "CODIGO_OCUPACAO",
    "DESCRICAO_OCUPACAO",
    "DATA_NASCIMENTO",
    "NUM_TITULO_ELEITORAL_CANDIDATO",
    "IDADE_DATA_ELEICAO",
    "CODIGO_SEXO",
    "DESCRICAO_SEXO",
    "COD_GRAU_INSTRUCAO",
    "DESCRICAO_GRAU_INSTRUCAO",
    "CODIGO_ESTADO_CIVIL",
    "DESCRICAO_ESTADO_CIVIL",
    "CODIGO_COR_RACA",
    "DESCRICAO_COR_RACA",
    "CODIGO_NACIONALIDADE",
    "DESCRICAO_NACIONALIDADE",
    "SIGLA_UF_NASCIMENTO",
    "CODIGO_MUNICIPIO_NASCIMENTO",
    "NOME_MUNICIPIO_NASCIMENTO",
    "DESPESA_MAX_CAMPANHA",
    "COD_SIT_TOT_TURNO",
    "DESC_SIT_TOT_TURNO",
    "NM_EMAIL",
]

Feito isso, vamos criar um pandas DataFrame com todos os dados concatenados.

_Obs.: Será necessário tratar os dados das eleições 2018 quando o dataset for liberado pelo TSE_

In [42]:
cand_df = pd.DataFrame()
for ano in anos_eleitorais:
    arquivos = PREFIXO_ARQUIVO_CANDIDATURAS + ano + '*.txt'
    pathes = os.path.join(DIRETORIO_TEMPORARIO, arquivos)
    arquivos_do_ano = sorted(glob.glob(pathes))
    for arquivo_i in arquivos_do_ano:
        if ('2014' in arquivo_i) or ('2016' in arquivo_i):
            cand_df_i = pd.read_csv(
                arquivo_i,
                sep=';',
                header=None,
                dtype=np.str,
                names=header_consulta_cand_pos2014,
                encoding='iso-8859-1')
        elif ('2012' in arquivo_i):
            cand_df_i = pd.read_csv(
                arquivo_i,
                sep=';',
                header=None,
                dtype=np.str,
                names=header_consulta_cand_em2012,
                encoding='iso-8859-1')
        else:
            cand_df_i = pd.read_csv(
                arquivo_i,
                sep=';',
                header=None,
                dtype=np.str,
                names=header_consulta_cand_ate2010,
                encoding='iso-8859-1')
        cand_df = cand_df.append(cand_df_i)
        print(ano + arquivo_i + ' arquivo anexado com sucesso!')

2004../temp\consulta_cand_2004_AC.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_AL.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_AM.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_AP.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_BA.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_CE.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_ES.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_GO.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_MA.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_MG.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_MS.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_MT.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_PA.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_PB.txt arquivo anexado com sucesso!
2004../temp\consulta_cand_2004_PE.txt arquivo anexado com suce

2012../temp\consulta_cand_2012_PI.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_PR.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_RJ.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_RN.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_RO.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_RR.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_RS.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_SC.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_SE.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_SP.txt arquivo anexado com sucesso!
2012../temp\consulta_cand_2012_TO.txt arquivo anexado com sucesso!
2014../temp\consulta_cand_2014_AC.txt arquivo anexado com sucesso!
2014../temp\consulta_cand_2014_AL.txt arquivo anexado com sucesso!
2014../temp\consulta_cand_2014_AM.txt arquivo anexado com sucesso!
2014../temp\consulta_cand_2014_AP.txt arquivo anexado com suce

In [43]:
cand_df

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
0,2004,11,NaN,3,5,-1,1,601,2,4,...,13,13,000356682496,1,6,#NE#,PT,01120,AC,RS
1,2004,11,NaN,3,1,-1,1,601,2,3,...,14,14,000525772445,1,1,#NE#,PTB,01120,AC,AC
2,2004,11,NaN,3,12,-1,1,143,2,8,...,45,45,002053882488,1,27,#NE#,PSDB,01120,AC,PR
3,2004,12,NaN,3,7,-1,1,601,2,0,...,13,40,002817512496,1,7,#NE#,PSB,01120,AC,AC
4,2004,12,NaN,5,3,-1,1,143,2,8,...,14,14,002600402445,1,2,#NE#,PTB,01120,AC,AC
5,2004,12,NaN,3,14,-1,1,999,2,4,...,45,15,000411782470,1,28,#NE#,PMDB,01120,AC,PR
6,2004,13,NaN,3,18,-1,1,999,2,2,...,15120,15,001022542470,1,43,#NE#,PMDB,01120,AC,PR
7,2004,13,NaN,3,10,-1,1,601,2,3,...,13113,13,001750702445,1,23,#NE#,PT,01120,AC,ES
8,2004,13,NaN,3,10,-1,1,601,2,2,...,13613,13,001580082461,1,18,#NE#,PT,01120,AC,PR
9,2004,13,NaN,1,9,-1,1,999,2,4,...,43333,43,003552582461,1,8,#NE#,PV,01120,AC,RO


Como o _index_ não contém não informação útil, vamos resetá-lo

In [44]:
# this index contains no useful information
cand_df.index = cand_df.reset_index().index

In [45]:
cand_df

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
0,2004,11,NaN,3,5,-1,1,601,2,4,...,13,13,000356682496,1,6,#NE#,PT,01120,AC,RS
1,2004,11,NaN,3,1,-1,1,601,2,3,...,14,14,000525772445,1,1,#NE#,PTB,01120,AC,AC
2,2004,11,NaN,3,12,-1,1,143,2,8,...,45,45,002053882488,1,27,#NE#,PSDB,01120,AC,PR
3,2004,12,NaN,3,7,-1,1,601,2,0,...,13,40,002817512496,1,7,#NE#,PSB,01120,AC,AC
4,2004,12,NaN,5,3,-1,1,143,2,8,...,14,14,002600402445,1,2,#NE#,PTB,01120,AC,AC
5,2004,12,NaN,3,14,-1,1,999,2,4,...,45,15,000411782470,1,28,#NE#,PMDB,01120,AC,PR
6,2004,13,NaN,3,18,-1,1,999,2,2,...,15120,15,001022542470,1,43,#NE#,PMDB,01120,AC,PR
7,2004,13,NaN,3,10,-1,1,601,2,3,...,13113,13,001750702445,1,23,#NE#,PT,01120,AC,ES
8,2004,13,NaN,3,10,-1,1,601,2,2,...,13613,13,001580082461,1,18,#NE#,PT,01120,AC,PR
9,2004,13,NaN,1,9,-1,1,999,2,4,...,43333,43,003552582461,1,8,#NE#,PV,01120,AC,RO


Agora, exportaremos o conjunto completo para um super `.csv` comprimido em `xz`

In [46]:
cand_df.to_csv(
    OUTPUT_DATASET_PATH,
    encoding='utf-8',
    compression='xz',
    header=True,
    index=False)

## Tratar os Dados

### Estados

In [127]:
estados_df = cand_df[[
    'SIGLA_UF'
]].drop_duplicates('SIGLA_UF', keep='last').sort_values(by=['SIGLA_UF'], ascending=True, na_position='first')

print(estados_df)
print(estados_df.shape)

        SIGLA_UF
1342066       AC
1349338       AL
1358906       AM
1360796       AP
1397606       BA
1317068       BR
1412524       CE
1319152       DF
1423018       ES
1444280       GO
1462807       MA
1541596       MG
1548992       MS
1559423       MT
1580045       PA
1592042       PB
1611183       PE
1621381       PI
1652767       PR
1675064       RJ
1684194       RN
1688873       RO
1690662       RR
1719569       RS
1737236       SC
1743134       SE
1829780       SP
1837258       TO
(28, 1)


In [128]:
estados = pd.DataFrame({
    'sigla': [
        'AC',
        'AL',
        'AM',
        'AP',
        'BA',
        'CE',
        'DF',
        'ES',
        'GO',
        'MA',
        'MG',
        'MS',
        'MT',
        'PA',
        'PB',
        'PE',
        'PI',
        'PR',
        'RJ',
        'RN',
        'RO',
        'RR',
        'RS',
        'SC',
        'SE',
        'SP',
        'TO'
    ],
    'nome': [
        'Acre',
        'Alagoas',
        'Amazonas',
        'Amapá',
        'Bahia',
        'Ceará',
        'Distrito Federal',
        'Espírito Santos',
        'Goiás',
        'Maranhão',
        'Minas Gerais',
        'Mato Grosso do Sul',
        'Mato Grosso',
        'Pará',
        'Paraíba',
        'Pernambuco',
        'Piauí',
        'Paraná',
        'Rio de Janeiro',
        'Rio Grande do Norte',
        'Rondônia',
        'Roraima',
        'Rio Grande do Sul',
        'Santa Catarina',
        'Sergipe',
        'São Paulo',
        'Tocantins'
    ]
})

print(estados)
print(estados.shape)

                   nome sigla
0                  Acre    AC
1               Alagoas    AL
2              Amazonas    AM
3                 Amapá    AP
4                 Bahia    BA
5                 Ceará    CE
6      Distrito Federal    DF
7       Espírito Santos    ES
8                 Goiás    GO
9              Maranhão    MA
10         Minas Gerais    MG
11   Mato Grosso do Sul    MS
12          Mato Grosso    MT
13                 Pará    PA
14              Paraíba    PB
15           Pernambuco    PE
16                Piauí    PI
17               Paraná    PR
18       Rio de Janeiro    RJ
19  Rio Grande do Norte    RN
20             Rondônia    RO
21              Roraima    RR
22    Rio Grande do Sul    RS
23       Santa Catarina    SC
24              Sergipe    SE
25            São Paulo    SP
26            Tocantins    TO
(27, 2)


### Cidades

_Obs.: Foi necessário remover os últimos 28 resultados pois eles se referem a ESTADOS e não a CIDADES_

In [88]:
cidades = (
    cand_df[[
        'SIGLA_UE',
        'DESCRICAO_UE',
        'SIGLA_UF'
    ]]
    .drop_duplicates('SIGLA_UE', keep='last')
    .sort_values(by=['SIGLA_UE'], ascending=True, na_position='first')
    .rename(columns = {
        'SIGLA_UE': 'codigo_tse',
        'DESCRICAO_UE': 'nome',
        'SIGLA_UF': 'estado'
    })
)[:-28]

print(cidades.shape)
print(cidades)

(5568, 3)
        codigo_tse                       nome estado
1688797      00019              GUAJARÁ-MIRIM     RO
1688872      00035                PORTO VELHO     RO
1688864      00051                  JI-PARANÁ     RO
1688740      00078                  ARIQUEMES     RO
1688856      00094                     CACOAL     RO
1688705      00116              PIMENTA BUENO     RO
1688826      00132                    VILHENA     RO
1688704      00159                       JARU     RO
1688865      00175        OURO PRETO DO OESTE     RO
1688868      00191          PRESIDENTE MÉDICI     RO
1688812      00213              COSTA MARQUES     RO
1688866      00230          COLORADO DO OESTE     RO
1688871      00256           ESPIGÃO DO OESTE     RO
1688850      00272                 CEREJEIRAS     RO
1688821      00299             ROLIM DE MOURA     RO
1688828      00310     ALTA FLORESTA DO OESTE     RO
1688858      00337          ALVORADA DO OESTE     RO
1688504      00353       SANTA LUZIA

### Nacionalidades

In [124]:
nacionalidades = (
    cand_df[[
        'CODIGO_NACIONALIDADE',
        'DESCRICAO_NACIONALIDADE'
    ]]
    .drop_duplicates('CODIGO_NACIONALIDADE', keep='last')
    .sort_values(by=['CODIGO_NACIONALIDADE'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_NACIONALIDADE': 'codigo_tse',
        'DESCRICAO_NACIONALIDADE': 'nome'
    })
)[:-1]

print(nacionalidades.shape)
print(nacionalidades)

(5, 2)
        codigo_tse                                  nome
401701           0                         NÃO INFORMADA
1837258          1                       BRASILEIRA NATA
1836773          2             BRASILEIRA (NATURALIZADA)
1830351          3  PORTUGUESA COM IGUALDADE DE DIREITOS
1836916          4                           ESTRANGEIRO


Foi necessário remover a última entrada por se tratar de uma distorção, a entrada é incompatível com o tipo de dado:
`CODIGO_NACIONALIDADE = 'BRASILEIRA NATA'`

In [125]:
cand_df.loc[cand_df['CODIGO_NACIONALIDADE'] == 'BRASILEIRA NATA']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


### Graus de Instrução

In [140]:
graus_instrucao = (
    cand_df[[
        'COD_GRAU_INSTRUCAO',
        'DESCRICAO_GRAU_INSTRUCAO'
    ]]
    .drop_duplicates('COD_GRAU_INSTRUCAO', keep='last')
    .sort_values(by=['COD_GRAU_INSTRUCAO'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_GRAU_INSTRUCAO': 'codigo_tse',
        'DESCRICAO_GRAU_INSTRUCAO': 'descricao'
    })
)[:-2]

print(graus_instrucao.shape)
print(graus_instrucao)

(9, 2)
        codigo_tse                      descricao
410000           0                  NÃO INFORMADO
1790642          1                     ANALFABETO
1837248          2                   LÊ E ESCREVE
1837257          3  ENSINO FUNDAMENTAL INCOMPLETO
1837254          4    ENSINO FUNDAMENTAL COMPLETO
1837243          5        ENSINO MÉDIO INCOMPLETO
1837258          6          ENSINO MÉDIO COMPLETO
1837250          7            SUPERIOR INCOMPLETO
1837255          8              SUPERIOR COMPLETO


Foi necessário remover as duas últimas entrada por se tratar de uma distorção, a entrada é incompatível com o tipo de dado. Foram as mesmas entradas que deram problema na etapa de nacionalidade

In [138]:
cand_df.loc[cand_df['COD_GRAU_INSTRUCAO'] == 'ENSINO MÉDIO COMPLETO']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1


In [139]:
cand_df.loc[cand_df['COD_GRAU_INSTRUCAO'] == 'SUPERIOR COMPLETO']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


### Ocupações

In [144]:
ocupacoes = (
    cand_df[[
        'CODIGO_OCUPACAO',
        'DESCRICAO_OCUPACAO'
    ]]
    .drop_duplicates('CODIGO_OCUPACAO', keep='last')
    .sort_values(by=['CODIGO_OCUPACAO'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_OCUPACAO': 'codigo_tse',
        'DESCRICAO_OCUPACAO': 'descricao'
    })
)[:-2]

print(ocupacoes.shape)
print(ocupacoes)

(260, 2)
        codigo_tse                                          descricao
401787           0                                      NÃO INFORMADA
1836784        101                                         ENGENHEIRO
1834397        102                                          ARQUITETO
1836343        103                                           AGRÔNOMO
1829197        104                                            QUÍMICO
1792908        106                                             FÍSICO
1807883        107                                            GEÓLOGO
1837224        109                      AGENTE DE SAÚDE E SANITARISTA
1834161        110                                      AGENTE POSTAL
1837247        111                                             MÉDICO
1836577        112                                        VETERINÁRIO
1837189        113                                         ENFERMEIRO
1836494        114             FISIOTERAPEUTA E TERAPEUTA OCUPACIONAL
1837053    

Foi necessário remover as duas últimas entrada por se tratar de uma distorção, a entrada é incompatível com o tipo de dado.

In [157]:
cand_df.loc[cand_df['DESCRICAO_OCUPACAO'] == '18-DEC-50']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


In [143]:
cand_df.loc[cand_df['DESCRICAO_OCUPACAO'] == '29-DEC-69']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1


### Eleições

In [146]:
eleicoes = (
    cand_df[[
        'ANO_ELEICAO',
        'DESCRICAO_ELEICAO'
    ]]
    .drop_duplicates('DESCRICAO_ELEICAO', keep='last')
    .sort_values(by=['ANO_ELEICAO'], ascending=True, na_position='first')
    .rename(columns = {
        'ANO_ELEICAO': 'ano',
        'DESCRICAO_ELEICAO': 'descricao'
    })
)

print(eleicoes.shape)
print(eleicoes)

(201, 2)
          ano                               descricao
401788   2004                           ELEICOES 2004
422578   2006                           ELEICOES 2006
578196   2008                  ELEIÇÕES SUPLEMENTARES
800173   2008                              PLEBISCITO
807094   2008                           ELEIÇÕES 2008
807109   2008             ELEIÇÕES SUPLEMENTARES 2008
829686   2010                           ELEIÇÕES 2010
1200422  2012              MAJORITÁRIA - PEDRAS ALTAS
1200426  2012                   MAJORITÁRIA - TAQUARI
1200434  2012         MAJORITÁRIA - EUGÊNIO DE CASTRO
1200437  2012                   MAJORITÁRIA - ERECHIM
1200440  2012             MAJORITÁRIA - NOVO HAMBURGO
1200444  2012    MAJORITÁRIA - MAXIMILIANO DE ALMEIDA
1200456  2012          MAJORITÁRIA - SÃO JOSÉ DO OURO
1200452  2012      MAJORITÁRIA - SÃO JOSÉ DAS MISSÕES
1200418  2012                MAJORITÁRIA - SOBRADINHO
1200464  2012           MAJORITÁRIA - AUGUSTO PESTANA
1200468  2012      

### Cargos

In [152]:
cargos = (
    cand_df[[
        'CODIGO_CARGO',
        'DESCRICAO_CARGO'
    ]]
    .drop_duplicates('CODIGO_CARGO', keep='last')
    .sort_values(by=['CODIGO_CARGO'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_CARGO': 'codigo_tse',
        'DESCRICAO_CARGO': 'descricao'
    })
)[:-1]

print(cargos.shape)
print(cargos)

(13, 2)
        codigo_tse           descricao
1317068          1          PRESIDENTE
1339752         10         2º SUPLENTE
1837255         11            PREFEITO
1837246         12       VICE-PREFEITO
1837258         13            VEREADOR
1317066          2     VICE-PRESIDENTE
1339715          3          GOVERNADOR
1339703          4     VICE-GOVERNADOR
1339717          5             SENADOR
1339756          6    DEPUTADO FEDERAL
1339755          7   DEPUTADO ESTADUAL
1319152          8  DEPUTADO DISTRITAL
1339685          9         1º SUPLENTE


A última entrada foi removida da lista de cargos pois se refere a perguntas realizadas em plebiscitos

In [153]:
cand_df.loc[cand_df['CODIGO_CARGO'] == '91'][['DESCRICAO_CARGO']].values

array([['VOCÊ É A FAVOR | DA CRIAÇÃO DO MUNICÍPIO DE | EXTREMA DE RONDÔNIA?'],
       ['VOCÊ É A FAVOR | DA CRIAÇÃO DO MUNICÍPIO DE | EXTREMA DE RONDÔNIA?'],
       ['VOCÊ É A FAVOR DA | ALTERAÇÃO DO NOME | DA CIDADE DE EMBU" | PARA "EMBU DAS ARTES"?"'],
       ['VOCÊ É A FAVOR DA | ALTERAÇÃO DO NOME | DA CIDADE DE EMBU" | PARA "EMBU DAS ARTES"?"']],
      dtype=object)

### Partidos

In [155]:
partidos = (
    cand_df[[
        'NUMERO_PARTIDO',
        'SIGLA_PARTIDO',
        'NOME_PARTIDO'
    ]]
    .drop_duplicates('NUMERO_PARTIDO', keep='last')
    .sort_values(by=['NUMERO_PARTIDO'], ascending=True, na_position='first')
    .rename(columns = {
        'NUMERO_PARTIDO': 'numero',
        'SIGLA_PARTIDO': 'sigla',
        'NOME_PARTIDO': 'nome',
    })
)[:-2]

print(partidos.shape)
print(partidos)

(37, 3)
        numero    sigla                                            nome
1837240     10      PRB                  PARTIDO REPUBLICANO BRASILEIRO
1837255     11       PP                            PARTIDO PROGRESSISTA
1837229     12      PDT                 PARTIDO DEMOCRÁTICO TRABALHISTA
1837253     13       PT                       PARTIDO DOS TRABALHADORES
1837256     14      PTB                  PARTIDO TRABALHISTA BRASILEIRO
1837257     15     PMDB     PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO
1824733     16     PSTU  PARTIDO SOCIALISTA DOS TRABALHADORES UNIFICADO
1837217     17      PSL                          PARTIDO SOCIAL LIBERAL
1836945     18     REDE                           REDE SUSTENTABILIDADE
1837140     19      PTN                    PARTIDO TRABALHISTA NACIONAL
1837236     20      PSC                          PARTIDO SOCIAL CRISTÃO
1830854     21      PCB                    PARTIDO COMUNISTA BRASILEIRO
1837254     22       PR                            PARTI

Foi necessário remover as duas últimas entrada por se tratar de uma distorção, a entrada é incompatível com o tipo de dado.

In [158]:
cand_df.loc[cand_df['NUMERO_PARTIDO'] == 'PPS']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1


In [159]:
cand_df.loc[cand_df['NUMERO_PARTIDO'] == 'PR']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


### Sitações de Candidatura

In [165]:
situacoes_candidatura = (
    cand_df[[
        'COD_SITUACAO_CANDIDATURA',
        'DES_SITUACAO_CANDIDATURA'
    ]]
    .drop_duplicates('COD_SITUACAO_CANDIDATURA', keep='last')
    .sort_values(by=['COD_SITUACAO_CANDIDATURA'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_SITUACAO_CANDIDATURA': 'codigo_tse',
        'DES_SITUACAO_CANDIDATURA': 'descricao'
    })
)[:-1]

print(situacoes_candidatura.shape)
print(situacoes_candidatura)

(15, 2)
        codigo_tse                   descricao
1826796         10                     CASSADO
616879          11                   IMPUGNADO
1836321         13  NÃO CONHECIMENTO DO PEDIDO
1837252         14                  INDEFERIDO
1837030         16        DEFERIDO COM RECURSO
1805071         17      PENDENTE DE JULGAMENTO
1828962         18         CASSADO COM RECURSO
1684173         19       CANCELADO COM RECURSO
1837258          2                    DEFERIDO
1836955          4      INDEFERIDO COM RECURSO
1836637          5                   CANCELADO
1837240          6                    RENÚNCIA
1836293          7                    FALECIDO
1200484          8       AGUARDANDO JULGAMENTO
806347           9                  INELEGÍVEL


Foi necessário remover a última entrada por se tratar de uma distorção, a entrada é incompatível com o tipo de dado.

In [163]:
cand_df.loc[cand_df['COD_SITUACAO_CANDIDATURA'] == 'DEFERIDO']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


### Resultados de Candidatura

In [181]:
resultados_candidatura = (
    cand_df[[
        'COD_SIT_TOT_TURNO',
        'DESC_SIT_TOT_TURNO'
    ]]
    .drop_duplicates('COD_SIT_TOT_TURNO', keep='last')
    .sort_values(by=['COD_SIT_TOT_TURNO'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_SIT_TOT_TURNO': 'codigo_tse',
        'DESC_SIT_TOT_TURNO': 'sigla'
    })
)[3:15]

print(resultados_candidatura.shape)
print(resultados_candidatura)

(12, 2)
        codigo_tse                                         sigla
1837255          1                                        ELEITO
826076          10                                   SUBSTITUÍDO
829630          11                        INDEFERIDO COM RECURSO
807433          12                           CASSADO COM RECURSO
1837251          2                                 ELEITO POR QP
1837219          3                              ELEITO POR MÉDIA
1837246          4                                    NÃO ELEITO
1837258          5                                      SUPLENTE
1829480          6                                      2º TURNO
825631           7  RENÚNCIA/FALECIMENTO/CASSAÇÃO APÓS A ELEIÇÃO
829639           8              REGISTRO NEGADO ANTES DA ELEIÇÃO
820781           9                REGISTRO NEGADO APÓS A ELEIÇÃO


Além das entradas problemáticas, neste caso temos algumas situações:

- Campos preenchidos com '#NULO' significam que a informação está em branco no banco de dados. O correspondente para '#NULO' nos campos numéricos é '-1';
- Campos preenchidos com '#NE' significam que naquele ano a informação não era registrada em banco de dados pelos sistemas eleitorais. O correspondente para #NE nos campos numéricos é '-3';

In [173]:
cand_df.loc[cand_df['COD_SIT_TOT_TURNO'] == 'NaN']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO


In [171]:
cand_df.loc[cand_df['COD_SIT_TOT_TURNO'] == '-1']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
3,2004,12,NaN,3,7,-1,1,601,2,0,...,13,40,002817512496,1,7,#NE#,PSB,01120,AC,AC
4,2004,12,NaN,5,3,-1,1,143,2,8,...,14,14,002600402445,1,2,#NE#,PTB,01120,AC,AC
5,2004,12,NaN,3,14,-1,1,999,2,4,...,45,15,000411782470,1,28,#NE#,PMDB,01120,AC,PR
14,2004,13,NaN,1,18,-1,1,999,4,3,...,15123,15,020802772232,1,42,#NE#,PMDB,01120,AC,MS
30,2004,13,NaN,1,9,-1,1,999,2,6,...,40640,40,001867852470,1,17,#NE#,PSB,01120,AC,PR
38,2004,13,NaN,3,10,-1,1,999,2,3,...,13124,13,002815132437,1,22,#NE#,PT,01120,AC,PR
43,2004,13,NaN,3,18,-1,1,999,2,3,...,25678,25,002463832305,1,39,#NE#,PFL,01120,AC,MG
51,2004,12,NaN,3,5,-1,1,297,2,6,...,13,40,000141032488,1,34,#NE#,PSB,01570,AC,AC
52,2004,12,NaN,3,2,-1,1,143,4,6,...,23,25,000139702402,1,11,#NE#,PFL,01570,AC,AC
83,2004,13,NaN,1,3,-1,1,581,4,4,...,25678,25,001773602470,1,19,#NE#,PFL,01570,AC,AC


In [172]:
cand_df.loc[cand_df['COD_SIT_TOT_TURNO'] == '-3']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
8550,2004,12,NaN,9,29,-1,1,395,2,7,...,40,40,008895691740,2,139,#NE#,PSB,27855,AL,RJ
8551,2004,12,NaN,3,18,-1,1,102,4,8,...,12,14,000992611759,2,63,#NE#,PTB,27855,AL,PE
14697,2004,12,NaN,9,17,-1,1,201,2,8,...,40,12,004865942240,2,167,#NE#,PDT,02550,AM,CE
14698,2004,12,NaN,3,31,-1,1,201,2,6,...,25,23,003343832291,2,895,#NE#,PPS,02550,AM,AM
46637,2004,12,NaN,3,31,-1,1,131,2,8,...,12,45,008127370590,2,822,#NE#,PSDB,38490,BA,BA
46638,2004,12,NaN,3,19,-1,1,201,2,8,...,25,25,000643610507,2,176,#NE#,PFL,38490,BA,BA
58599,2004,12,NaN,3,63,-1,1,0,2,7,...,13,40,032233350736,2,1129,#NE#,PSB,13897,CE,PE
58600,2004,12,NaN,5,15,-1,1,143,4,8,...,25,26,007879160744,2,74,#NE#,PAN,13897,CE,CE
66611,2004,12,NaN,3,15,-1,1,125,2,6,...,13,40,015467331430,2,255,#NE#,PSB,56251,ES,ES
66612,2004,12,NaN,9,3,-1,1,111,2,8,...,45,15,000740321422,2,29,#NE#,PMDB,56251,ES,ES


In [179]:
cand_df.loc[cand_df['COD_SIT_TOT_TURNO'] == 'NÃO ELEITO']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
653018,2008,13,NaN,CASADO(A),#NE#,MANDAGUARI,BRASILEIRA NATA,"CARPINTEIRO, MARCENEIRO E ASSEMELHADOS",MASCULINO,ENSINO MÉDIO COMPLETO,...,23333,PPS,-1,1,30416,#NE#,PARTIDO POPULAR SOCIALISTA,76775,PR,-1


In [180]:
cand_df.loc[cand_df['COD_SIT_TOT_TURNO'] == 'SUPLENTE']

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
743740,2008,13,NaN,SEPARADO(A) JUDICIALMENTE,#NE#,LINS,BRASILEIRA NATA,APOSENTADO (EXCETO SERVIDOR PÚBLICO),FEMININO,SUPERIOR COMPLETO,...,22183,PR,-1,1,19494,#NE#,PARTIDO DA REPÚBLICA,62499,SP,-1


### Candidatos

In [182]:
candidatos = (
    cand_df[[
        'NOME_CANDIDATO',
        'DATA_NASCIMENTO',
        'CPF_CANDIDATO',
        'NUM_TITULO_ELEITORAL_CANDIDATO',
        'NM_EMAIL',
        'NOME_MUNICIPIO_NASCIMENTO',
        'CODIGO_NACIONALIDADE',
        'COD_GRAU_INSTRUCAO',
        'CODIGO_OCUPACAO',
    ]]
    .drop_duplicates('CPF_CANDIDATO', keep='last')
    # .sort_values(by=['NOME_CANDIDATO'], ascending=True, na_position='first')
    .rename(columns = {
        'NOME_CANDIDATO': 'nome',
        'DATA_NASCIMENTO': 'data_nascimento',
        'CPF_CANDIDATO': 'cpf',
        'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
        'NM_EMAIL': 'email',
        'NOME_MUNICIPIO_NASCIMENTO': 'cidade',
        'CODIGO_NACIONALIDADE': 'nacionalidade',
        'COD_GRAU_INSTRUCAO': 'grau_instrucao',
        'CODIGO_OCUPACAO': 'ocupacao'
    })
)

print(candidatos.shape)
print(candidatos)

(1257905, 9)
                                         nome data_nascimento          cpf  \
0                             JOÃO DE ALMEIDA        26081957  18139124249   
1              PAULO CÉSAR FERREIRA DE ARAUJO        08041957  07857136204   
3                    ERAIDES CAETANO DE SOUZA        26081957  40917860900   
12                        DAMARIS DE OLIVEIRA        24091966  49488929200   
14                     EDILENE SILVA DOS REIS        07041971  38950707268   
16                    EDNA BERNARDINO E SILVA        29061958  34006451253   
18                  ERACIDES CAETANO DE SOUZA        14011950  17630207972   
19                  ESTEVÃO DE SOUZA FERREIRA        26121978  61072621215   
20            FRANCISCO ANTONIO DA SILVA DIAS        15061964  21580430244   
23                     IVAN DA SILVA FERREIRA        23041974  44378793253   
26          JOSÉ GILDICELLI TEIXEIRA DA SILVA        14071955  19688440230   
27              JOSÉ IDNILTON SANTOS DA SILVA      

Existem 3 tipos diferentes de data de nascimento que devem ser tratados antes da inserção no banco de dados

In [194]:
tipos_data = (
    cand_df[[
        'ANO_ELEICAO',
        'DATA_NASCIMENTO',
    ]]
    .drop_duplicates('ANO_ELEICAO', keep='last')
)

print(tipos_data)

        ANO_ELEICAO DATA_NASCIMENTO
401788         2004        01061949
422578         2006      08/04/1945
807109         2008       26-OCT-59
829686         2010       03-JAN-34
1313511        2012      15/02/1958
1339756        2014      12/10/1965
1837258        2016      06/08/1969


### Candidaturas

In [196]:
candidaturas = (
    cand_df[[
        'NUMERO_CANDIDATO',
        'NOME_URNA_CANDIDATO',
        'NOME_LEGENDA',
        'COMPOSICAO_LEGENDA',
        'DESPESA_MAX_CAMPANHA',

        'CPF_CANDIDATO', # candidato_id
        'SIGLA_UE', # cidade_id
        'SIGLA_UF', # estado_id
        'NUMERO_PARTIDO', # partido_id
        'CODIGO_CARGO', # cargo_id
        'COD_SITUACAO_CANDIDATURA', # situacao_candidatura_id
        'COD_SIT_TOT_TURNO', # resultado_id
    ]]
    .rename(columns = {
        'NUMERO_CANDIDATO': 'numero_candidato',
        'NOME_URNA_CANDIDATO': 'nome_urna',
        'NOME_LEGENDA': 'nome_legenda',
        'COMPOSICAO_LEGENDA': 'composicao_legenda',
        'DESPESA_MAX_CAMPANHA': 'despesa_maxima',

        'CPF_CANDIDATO': 'candidato_cpf', # candidato_id
        'SIGLA_UE': 'cidade', # cidade_id
        'SIGLA_UF': 'estado', # estado_id
        'NUMERO_PARTIDO': 'partido_numero', # partido_id
        'CODIGO_CARGO': 'cargo_codigo_tse', # cargo_id
        'COD_SITUACAO_CANDIDATURA': 'situacao_candidatura_codigo_tse', # situacao_candidatura_id
        'COD_SIT_TOT_TURNO': 'resultado_codigo_tse', # resultado_id
    })
)

print(candidaturas.shape)
print(candidaturas)

(1837259, 12)
        numero_candidato                nome_urna  \
0                     13                    JANGO   
1                     14                 PAULINHO   
2                     45             TIÃO BOCALOM   
3                     13                    GRAIA   
4                     14        PROFESSOR RICARDO   
5                     45               PR. VILSEU   
6                  15120                    ALAOR   
7                  13113                    VAUNA   
8                  13613                APARECIDO   
9                  43333                  ARISTON   
10                 14014                CARLINHOS   
11                 13456        PININHO CESALPINO   
12                 45123                  DAMARIS   
13                 25555                 DERMIVAL   
14                 15123                  EDILENE   
15                 13611                   TOBIAS   
16                 12345          PROFESSORA EDNA   
17                 45678        

## A Ser Continuado

Antes de partir para o povoamento do banco de dados, é preciso refletir sobre qual a melhor maneira de armazenar os dados de composição de legenda

#### Links úteis
https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/

In [195]:
cand_df[['SIGLA_LEGENDA','COMPOSICAO_LEGENDA','NOME_LEGENDA']]

,SIGLA_LEGENDA,COMPOSICAO_LEGENDA,NOME_LEGENDA
0,#NE#,PT / PMN / PSB / PV / PC do B,FRENTE POPULAR DE ACRELÂNDIA
1,#NE#,#NE#,#NE#
2,#NE#,PP / PDT / PMDB / PPS / PFL / PSDB,CONTINUANDO O PROGRESSO
3,#NE#,PT / PMN / PSB / PV / PC do B,FRENTE POPULAR DE ACRELÂNDIA
4,#NE#,#NE#,#NE#
5,#NE#,PP / PDT / PMDB / PPS / PFL / PSDB,CONTINUANDO O PROGRESSO
6,#NE#,PP / PMDB / PPS / PFL,CONTINUANDO O PROGRESSO II
7,#NE#,#NE#,#NE#
8,#NE#,#NE#,#NE#
9,#NE#,PMN / PSB / PV / PC do B,UNIDOS PARA VENCER
